## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/Notebooks/prices.csv')

### Check all columns in the dataset

In [0]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date','symbol'],axis=1)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data= data.head(1000)

In [0]:
data.shape
features=data

In [0]:
label=data.pop('volume')

In [0]:
data.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [0]:
label=label.div(1000,000)

In [0]:
label.head()

0    2163.6
1    2386.4
2    2489.5
3    2006.3
4    1408.6
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x=data
y=label
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3, random_state=42)

In [0]:
x_test.shape
x_train.shape
y_train.shape
y_test.shape

x_train.dtypes

open     float32
close    float32
low      float32
high     float32
dtype: object

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
y_train=y_train.astype('float32')
y_test=y_test.astype('float32')


('x_test', open     float32
close    float32
low      float32
high     float32
dtype: object)


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import normalize

In [0]:
x_train = tf.math.l2_normalize(x_train)

In [0]:
x_test = tf.math.l2_normalize(x_test)
y_train = tf.math.l2_normalize(y_train)
y_test = tf.math.l2_normalize(y_test)
print("normzlized the train and test data sets")

normzlized the train and test data sets


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W=tf.zeros(shape=(5,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(features, W, b):
    y_pred = tf.add(tf.matmul(features, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
for i in range(100):    
    W, b = train(features, target, W, b)
    print('Current Training Loss on iteration', i, loss(target, prediction(features, W, b)).numpy())

AttributeError: ignored

### Get the shapes and values of W and b

In [0]:
W.numpy()

AttributeError: ignored

### Model Prediction on 1st Examples in Test Dataset

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data = pd.read_csv('/gdrive/My Drive/Notebooks/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
data.groupby(['Class']).count()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm)
Class,,,,
Iris-setosa,50,50,50,50
Iris-versicolor,50,50,50,50
Iris-virginica,50,50,50,50


In [0]:
X = data.drop(labels= "Class" , axis = 1)

In [0]:
y = data[["Class"]]

In [0]:
y = pd.get_dummies(y, drop_first=True)

In [0]:
y.head()


,Class_Iris-versicolor,Class_Iris-virginica
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
 #Initialize Sequential model
model = tf.keras.models.Sequential()

# Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((16,),input_shape=(4,)))

# Normalize the data
##model.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3,input_shape=(4,),activation='softmax'))

AttributeError: ignored

### Model Training 

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Prediction

In [0]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
          batch_size = X_train.shape[0])

ValueError: ignored

### Save the Model

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?